In [2]:
#Ici , on va étudier le succes critique a travers des critiques et des notes attribuées aux films

#On va d'abord charger le dataset original
import pandas as pd
df = pd.read_csv('movie_dataset_cleaned_final.csv')
print(df.head())

      budget                                    genres original_language  \
0  237000000  Action Adventure Fantasy Science Fiction                en   
1  300000000                  Adventure Fantasy Action                en   
2  245000000                    Action Adventure Crime                en   
3  250000000               Action Crime Drama Thriller                en   
4  260000000          Action Adventure Science Fiction                en   

   popularity                               production_companies  \
0  150.437577  [{"name": "Ingenious Film Partners", "id": 289...   
1  139.082615  [{"name": "Walt Disney Pictures", "id": 2}, {"...   
2  107.376788  [{"name": "Columbia Pictures", "id": 5}, {"nam...   
3  112.312950  [{"name": "Legendary Pictures", "id": 923}, {"...   
4   43.926995        [{"name": "Walt Disney Pictures", "id": 2}]   

                                production_countries     revenue  runtime  \
0  [{"iso_3166_1": "US", "name": "United States o...  278

In [3]:
#Maintenant on va le transformer en dataset supervisé pour un apprentissage machine-learning
#On ajoute une colonne "critique_success" qui vaut 1 si la note moyenne est supérieure à la moyenne générale des notes
df['critique_success'] = (df['vote_average'] > df['vote_average'].mean()).astype(int)
print(df[['vote_average', 'critique_success']].head())
#On sauvegarde le nouveau dataset
df.to_csv('movie_dataset_supervised_critique.csv', index=False)



   vote_average  critique_success
0           7.2                 1
1           6.9                 1
2           6.3                 1
3           7.6                 1
4           6.1                 1


In [4]:
#Maintenant on va supprimer les colonnes inutiles pour l'apprentissage supervisé
df = df.drop(columns=['genres', 'production_companies', 'production_countries', 'spoken_languages', 'cast'])
print(df.head())
#On sauvegarde le dataset nettoyé
df.to_csv('movie_dataset_supervised_critique_cleaned.csv', index=False)



      budget original_language  popularity     revenue  runtime  vote_average  \
0  237000000                en  150.437577  2787965087    162.0           7.2   
1  300000000                en  139.082615   961000000    169.0           6.9   
2  245000000                en  107.376788   880674609    148.0           6.3   
3  250000000                en  112.312950  1084939099    165.0           7.6   
4  260000000                en   43.926995   284139100    132.0           6.1   

   vote_count           director  critique_success  
0       11800      James Cameron                 1  
1        4500     Gore Verbinski                 1  
2        4466         Sam Mendes                 1  
3        9106  Christopher Nolan                 1  
4        2124     Andrew Stanton                 1  


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

# Chargement du dataset nettoyé
df = pd.read_csv('movie_dataset_supervised_critique_cleaned.csv')

# Features / target
X = df[['budget', 'original_language', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director']]
y = df['critique_success']

# Colonnes
categorical_cols = ['original_language', 'director']
numeric_cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']

# Preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Pipeline finale
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Entraînement
clf.fit(X_train, y_train)

# Prédictions
y_pred = clf.predict(X_test)

# Évaluation
print("Decision Tree Classifier Results:")
print(f"Accuracy: {clf.score(X_test, y_test):.2f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

#Score du modèle
correct_predictions = (y_test == y_pred).sum()
total_predictions = y_test.shape[0]
print(f"Nombre de prédictions correctes: {correct_predictions} sur {total_predictions}")
accuracy = correct_predictions / total_predictions
print(f"Précision du modèle: {accuracy:.2f}")

Decision Tree Classifier Results:
Accuracy: 1.00

Confusion Matrix:
[[1021    0]
 [   0 1381]]
Nombre de prédictions correctes: 2402 sur 2402
Précision du modèle: 1.00


In [23]:
# --- 1. Imports ---
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

# --- 2. Chargement du dataset ---
df = pd.read_csv('movie_dataset_supervised_critique_cleaned.csv')

# --- 3. Features / Target ---
X = df[['budget', 'original_language', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director']]
y = df['critique_success']

# --- 4. Colonnes numériques / catégoriques ---
categorical_cols = ['original_language', 'director']
numeric_cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']

# --- 5. Préprocessing ---

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# --- 6. Pipeline avec Forêt Aléatoire ---
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=100,       # nombre d'arbres
        max_depth=None,        # profondeur illimitée
        random_state=42,
        n_jobs=-1              # utilise tous les cœurs CPU
    ))
])

# --- 7. Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

# --- 8. Entraînement ---
clf.fit(X_train, y_train)

# --- 9. Prédictions ---
y_pred = clf.predict(X_test)

# --- 10. Évaluation ---
print("Random Forest Classifier Results:")
print(f"Accuracy: {clf.score(X_test, y_test):.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Score de précision
correct_predictions = (y_test == y_pred).sum()
accuracy = correct_predictions / len(y_test)
print(f"\nModel Accuracy: {accuracy:.2f}")


Random Forest Classifier Results:
Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1021
           1       1.00      1.00      1.00      1381

    accuracy                           1.00      2402
   macro avg       1.00      1.00      1.00      2402
weighted avg       1.00      1.00      1.00      2402


Confusion Matrix:
[[1021    0]
 [   0 1381]]

Model Accuracy: 1.00


In [19]:
# Appliquons le classifieur bayésien (Naive Bayes)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

# Chargement du dataset nettoyé
df = pd.read_csv('movie_dataset_supervised_critique_cleaned.csv')

# Supprimer les lignes avec des valeurs manquantes
df = df.dropna()

# Features / target
X = df[['budget', 'original_language', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director']]
y = df['critique_success']

# Colonnes catégoriques et numériques
categorical_cols = ['original_language', 'director']
numeric_cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']

# Preprocessing pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Preprocessing pour les colonnes catégoriques
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinaison des transformateurs
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Pipeline finale avec le classificateur Naive Bayes
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB())
])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Entraînement
clf.fit(X_train, y_train)

# Prédictions
y_pred = clf.predict(X_test)

# Évaluation
print("Gaussian Naive Bayes Classifier Results:")
print(f"Accuracy: {clf.score(X_test, y_test):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Score du modèle
correct_predictions = (y_test == y_pred).sum()
accuracy = correct_predictions / len(y_test)
print(f"\nModel Accuracy: {accuracy:.2f}")
print(f'Nombre de prédictions correctes: {correct_predictions} sur {len(y_test)}')

Gaussian Naive Bayes Classifier Results:
Accuracy: 0.50

Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.84      0.59      1017
           1       0.68      0.25      0.36      1369

    accuracy                           0.50      2386
   macro avg       0.57      0.55      0.48      2386
weighted avg       0.58      0.50      0.46      2386


Confusion Matrix:
[[ 856  161]
 [1028  341]]

Model Accuracy: 0.50
Nombre de prédictions correctes: 1197 sur 2386


In [21]:
# Appliquons K-Nearest Neighbors (KNN)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

# Chargement du dataset nettoyé
df = pd.read_csv('movie_dataset_supervised_critique_cleaned.csv')

# Supprimer les lignes avec des valeurs manquantes
df = df.dropna()

# Features / target
X = df[['budget', 'original_language', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director']]
y = df['critique_success']

# Colonnes catégoriques et numériques
categorical_cols = ['original_language', 'director']
numeric_cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']

# Preprocessing pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Preprocessing pour les colonnes catégoriques
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinaison des transformateurs
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Pipeline finale avec le classificateur KNN
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=5))
])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Entraînement
clf.fit(X_train, y_train)

# Prédictions
y_pred = clf.predict(X_test)

# Évaluation
print("K-Nearest Neighbors Classifier Results:")
print(f"Accuracy: {clf.score(X_test, y_test):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Score du modèle
correct_predictions = (y_test == y_pred).sum()
accuracy = correct_predictions / len(y_test)
print(f"\nModel Accuracy: {accuracy:.2f}")
print(f'Nombre de prédictions correctes: {correct_predictions} sur {len(y_test)}')

K-Nearest Neighbors Classifier Results:
Accuracy: 0.60

Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.47      0.51      1017
           1       0.64      0.70      0.67      1369

    accuracy                           0.60      2386
   macro avg       0.59      0.59      0.59      2386
weighted avg       0.60      0.60      0.60      2386


Confusion Matrix:
[[482 535]
 [409 960]]

Model Accuracy: 0.60
Nombre de prédictions correctes: 1442 sur 2386
